In [14]:
import subprocess
import sys
from functools import partial
import logging
from pathlib import Path
from time import strftime, gmtime
from typing import List

import numpy as np
import pandas as pd
from tqdm import tqdm

from ilids.utils.ffmpeg_tqdm import analyze_stream, ffmpeg_to_tqdm

In [8]:
def get_ffmpeg_select_argument(frames_sequence: List[int], setpts_filter: float) -> str:
    """Produce a string to be used with the FFMPEG command and its "select" argument.

    :param frames_sequence: linear or not list of frames to extract.
                            Example: 0, 12, 24, 36, 48, 60, ...
                            or:      6, 23, 27, 42, 50, 68, ...
    :param setpts_filter: from FFMPEG manual page:
                            "setpts" filter, which only sets timestamps and otherwise passes the frames unchanged
                          It is expected to be smaller than 1 in can less frames are selected and it is wished to
                          speed up the video.
                          Usually, it is expected to get 1/FRAMES_SKIP
                          Example: *0.25* (for a FRAMES_SKIP of 4)
    :return: string to be used to run the FFMPEG command for the "select" argument like: ffmpeg -i SZTRA201a08_224.mp4 -vf select='eq(n\,0)+eq(n\,12)+eq(n\,24)+eq(n\,36)+eq(n\,48)+eq(n\,60), setpts=0.08333333333*PTS' -an SZTRA201a08_224_12_random.mp4
    """
    # eq(n\\,0)+eq(n\\,12)+eq(n\\,24)+eq(n\\,36)+eq(n\\,48)
    eq_chain = "+".join([f"eq(n\,{n})" for n in frames_sequence])
    setpts = f"setpts={setpts_filter}*PTS"
    return f"'{eq_chain}',{setpts}"


def trivial(interval: int, frames_count: int):
    return list(range(0, frames_count, interval)) + (
        [] if (frames_count - 1) % interval == 0 else [frames_count - 1]
    )

In [23]:
# # Adapted from CLIP4Clip: https://github.com/ArrowLuo/CLIP4Clip/blob/master/preprocess/compress_video.py
# # but don't change the frame rate as be which to have different frame selection strategies
# # and use ffmpeg_to_tqdm helper
# def scale_compress_video(input_video_path: Path, output_video_path: Path) -> int:
#     logger = logging.getLogger(f"ffmpeg-{str(output_video_path)}")
#     logger.debug("Using ffmpeg to scale and compress an original video.")

#     stream_info = analyze_stream(logger, str(input_video_path))

#     command = [
#         "ffmpeg",
#         "-y",  # (optional) overwrite output file if it exists
#         "-i",
#         str(input_video_path),
#         "-filter:v",
#         "scale='if(gt(a,1),trunc(oh*a/2)*2,224)':'if(gt(a,1),224,trunc(ow*a/2)*2)'",  # scale to 224
#         "-map",
#         "0:v",
#         "-vcodec",
#         "libx264",  # additionally, change the codec for better
#         # compression
#         # display codecs with: `ffmpeg -codecs`
#         #    '-r', '3',  # frames per second
#         str(output_video_path),
#     ]
#     # to avoid carriage return ('\r') in ffmpeg output, to mess with the reading of the
#     # progress, use 'universal_newlines' argument
#     # https://github.com/chriskiehl/Gooey/issues/495#issuecomment-614991802
#     ffmpeg_process = subprocess.Popen(
#         command,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.STDOUT,
#         universal_newlines=True,
#     )

#     return ffmpeg_to_tqdm(
#         logger,
#         ffmpeg_process,
#         duration=stream_info.get("duration"),
#         tqdm_desc="FFMPEG scale down to 224 and encode libx264",
#     ).returncode

# # Using Stackoverflow answer, we can select multiple frames with the '+' "operation" inside the select
# def get_ffmpeg_select_argument(frames_sequence: List[int], setpts_filter: float) -> str:
#     """Produce a string to be used with the FFMPEG command and its "select" argument.

#     :param frames_sequence: linear or not list of frames to extract.
#                             Example: 0, 12, 24, 36, 48, 60, ...
#                             or:      6, 23, 27, 42, 50, 68, ...
#     :param setpts_filter: from FFMPEG manual page:
#                             "setpts" filter, which only sets timestamps and otherwise passes the frames unchanged
#                           It is expected to be smaller than 1 in can less frames are selected and it is wished to
#                           speed up the video.
#                           Usually, it is expected to get 1/FRAMES_SKIP
#                           Example: *0.25* (for a FRAMES_SKIP of 4)
#     :return: string to be used to run the FFMPEG command for the "select" argument like: ffmpeg -i SZTRA201a08_224.mp4 -vf select='eq(n\,0)+eq(n\,12)+eq(n\,24)+eq(n\,36)+eq(n\,48)+eq(n\,60), setpts=0.08333333333*PTS' -an SZTRA201a08_224_12_random.mp4
#     """
#     # eq(n\\,0)+eq(n\\,12)+eq(n\\,24)+eq(n\\,36)+eq(n\\,48)
#     eq_chain = "+".join([f"eq(n\,{n})" for n in frames_sequence])
#     setpts = f"setpts={setpts_filter}*PTS"
#     return f"'{eq_chain}',{setpts}"
#
# def extract_frames_in_video(
#     frames_sequence: List[int],
#     setpts_filter: float,
#     input_video_path: Path,
#     output_video_path: Path,
# ) -> int:
#     logger = logging.getLogger(f"ffmpeg-{str(output_video_path)}")
#     logger.setLevel(logging.INFO)
#     logger.debug("Using ffmpeg to extract frames from a video.")

#     stream_info = analyze_stream(logger, str(input_video_path))

#     command = [
#         "ffmpeg",
#         "-y",  # (optional) overwrite output file if it exists
#         "-i",
#         str(input_video_path),
#         "-vf",
#         f"select={get_ffmpeg_select_argument(frames_sequence, setpts_filter)}",
#         "-an",  # _The presence of "-an" disables audio stream selection for_ its output
#         str(output_video_path),
#     ]

#     # to avoid carriage return ('\r') in ffmpeg output, to mess with the reading of the
#     # progress, use 'universal_newlines' argument
#     # https://github.com/chriskiehl/Gooey/issues/495#issuecomment-614991802
#     ffmpeg_process = subprocess.Popen(
#         command,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.STDOUT,
#         universal_newlines=True,
#     )
#     return ffmpeg_to_tqdm(
#         logger,
#         ffmpeg_process,
#         duration=stream_info.get("duration"),
#         tqdm_desc="FFMPEG select subset of frames",
#     ).returncode


# Adapted from CLIP4Clip: https://github.com/ArrowLuo/CLIP4Clip/blob/master/preprocess/compress_video.py
# but don't change the frame rate as be which to have different frame selection strategies
# use ffmpeg_to_tqdm helper
# and directly extract a sequence of frames
def scale_compress_select_sequence(
    input_video_path: Path,
    output_video_path: Path,
    frames_sequence: List[int],
    setpts_filter: float,
) -> int:
    logger = logging.getLogger(f"ffmpeg-{str(output_video_path)}")
    logger.debug("Using ffmpeg to scale and compress and extract sub sequence")

    stream_info = analyze_stream(logger, str(input_video_path))

    command = [
        "ffmpeg",
        "-y",  # (optional) overwrite output file if it exists
        "-i",
        str(input_video_path),
        "-vf",
        "scale='if(gt(a,1),trunc(oh*a/2)*2,224)':'if(gt(a,1),224,trunc(ow*a/2)*2)'",  # scale to 224
        "-vf",
        f"select={get_ffmpeg_select_argument(frames_sequence, setpts_filter)}",  # select specific frames
        "-map",
        "0:v",
        "-vcodec",
        "libx264",  # additionally, change the codec for better
        "-an",  # Manual: The presence of "-an" disables audio stream selection for [its output]
        str(output_video_path),
    ]
    # to avoid carriage return ('\r') in ffmpeg output, to mess with the reading of the
    # progress, use 'universal_newlines' argument
    # https://github.com/chriskiehl/Gooey/issues/495#issuecomment-614991802
    ffmpeg_process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
    )

    ffmpeg_to_tqdm(
        logger,
        ffmpeg_process,
        duration=stream_info.get("duration"),
        tqdm_desc="FFMPEG scale down to 224, select frames and encode libx264",
    )

    ffmpeg_process.wait()

    return ffmpeg_process.returncode


In [10]:
short_df["StartFrame"] = short_df["StartTime"].dt.seconds * 25
short_df["EndFrame"] = short_df["EndTime"].dt.seconds * 25
short_df["DurationFrame"] = short_df["EndFrame"] - short_df["StartFrame"]
short_df

,filename,Duration,StartTime,EndTime,Distance,SubjectApproachType,SubjectDescription,SubjectOrientation,Classification,Stage
0,SZTR/video/SZTRA103a17.mov,36.4,0 days 00:00:09,0 days 00:00:36,10.0,Walk With Ladder,One person,Perpendicular,TP,1
1,SZTR/video/SZTRA203a17.mov,36.4,0 days 00:00:10,0 days 00:00:36,30.0,Walk With Ladder,One person,Perpendicular,TP,2
2,SZTR/video/SZTRA202b13.mov,39.2,0 days 00:00:19,0 days 00:00:39,10.0,Walk With Ladder,One person,Perpendicular,TP,2
3,SZTR/video/SZTRA102b13.mov,39.2,0 days 00:00:20,0 days 00:00:39,30.0,Walk With Ladder,One person,Perpendicular,TP,1
4,SZTR/video/SZTRA203b17.mov,44.32,0 days 00:00:13,0 days 00:00:42,30.0,Walk With Ladder,One person,Perpendicular,TP,2
5,SZTR/video/SZTRA103b17.mov,49.6,0 days 00:00:12,0 days 00:00:44,10.0,Walk With Ladder,One person,Perpendicular,TP,1
6,SZTR/video/SZTRA103b17.mov,49.6,0 days 00:00:00,0 days 00:00:45,None,None,None,None,FP,1
7,SZTR/video/SZTRA202a10.mov,82.72,0 days 00:00:06,0 days 00:01:21,None,None,None,None,FP,2
8,SZTR/video/SZTRA202a10.mov,82.72,0 days 00:00:08,0 days 00:00:52,30.0,Walk With Ladder,One person,Perpendicular,TP,2
9,SZTR/video/SZTRA202a10.mov,82.72,0 days 00:00:00,0 days 00:01:17,None,None,None,None,FP,2


In [11]:
short_df["OutFilename"] = short_df["filename"].apply(lambda f: str(Path(f).parent / Path(f).stem).replace("/", "_")) + "_" + short_df["StartTime"].dt.seconds.apply(lambda secs: strftime("%H_%M_%S", gmtime(secs))) + "_" + short_df["Classification"] + ".mp4"
short_df["OutFilename"].head()

,filename,Duration,StartTime,EndTime,Distance,SubjectApproachType,SubjectDescription,SubjectOrientation,Classification,Stage,StartFrame,EndFrame,DurationFrame
0,SZTR/video/SZTRA103a17.mov,36.4,0 days 00:00:09,0 days 00:00:36,10.0,Walk With Ladder,One person,Perpendicular,TP,1,225,900,675
1,SZTR/video/SZTRA203a17.mov,36.4,0 days 00:00:10,0 days 00:00:36,30.0,Walk With Ladder,One person,Perpendicular,TP,2,250,900,650
2,SZTR/video/SZTRA202b13.mov,39.2,0 days 00:00:19,0 days 00:00:39,10.0,Walk With Ladder,One person,Perpendicular,TP,2,475,975,500
3,SZTR/video/SZTRA102b13.mov,39.2,0 days 00:00:20,0 days 00:00:39,30.0,Walk With Ladder,One person,Perpendicular,TP,1,500,975,475
4,SZTR/video/SZTRA203b17.mov,44.32,0 days 00:00:13,0 days 00:00:42,30.0,Walk With Ladder,One person,Perpendicular,TP,2,325,1050,725
5,SZTR/video/SZTRA103b17.mov,49.6,0 days 00:00:12,0 days 00:00:44,10.0,Walk With Ladder,One person,Perpendicular,TP,1,300,1100,800
6,SZTR/video/SZTRA103b17.mov,49.6,0 days 00:00:00,0 days 00:00:45,None,None,None,None,FP,1,0,1125,1125
7,SZTR/video/SZTRA202a10.mov,82.72,0 days 00:00:06,0 days 00:01:21,None,None,None,None,FP,2,150,2025,1875
8,SZTR/video/SZTRA202a10.mov,82.72,0 days 00:00:08,0 days 00:00:52,30.0,Walk With Ladder,One person,Perpendicular,TP,2,200,1300,1100
9,SZTR/video/SZTRA202a10.mov,82.72,0 days 00:00:00,0 days 00:01:17,None,None,None,None,FP,2,0,1925,1925


In [12]:
trivial_twelve = partial(trivial, 12)

for _, row in tqdm(short_df.iterrows(), desc="Extract sequences"):
    print(Path("..") / row["filename"], Path("..") / "data" / "sequences" / row["OutFilename"], (row["StartFrame"] + np.array(trivial_twelve(row["DurationFrame"]))).tolist(), 1 / 12)
    scale_compress_select_sequence(Path("..") / row["filename"], Path("..") / "data" / "sequences" / row["OutFilename"], (row["StartFrame"] + np.array(trivial_twelve(row["DurationFrame"]))).tolist(), 1 / 12)


0    SZTR_video_SZTRA103a17_00_00_09_TP.mp4
1    SZTR_video_SZTRA203a17_00_00_10_TP.mp4
2    SZTR_video_SZTRA202b13_00_00_19_TP.mp4
3    SZTR_video_SZTRA102b13_00_00_20_TP.mp4
4    SZTR_video_SZTRA203b17_00_00_13_TP.mp4
Name: OutFilename, dtype: object

In [24]:
trivial_twelve = partial(trivial, 12)

# root = logging.getLogger()
# root.setLevel(logging.INFO)
#
# handler = logging.StreamHandler(sys.stdout)
# handler.setLevel(logging.INFO)
# formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# handler.setFormatter(formatter)
# root.addHandler(handler)

for _, row in tqdm(short_df.iterrows(), desc="Extract sequences"):
    print(Path("..") / row["filename"], Path("..") / "data" / "sequences" / row["OutFilename"], (row["StartFrame"] + np.array(trivial_twelve(row["DurationFrame"]))).tolist(), 1 / 12)
    result = scale_compress_select_sequence(Path("..") / row["filename"], Path("..") / "data" / "sequences" / row["OutFilename"], (row["StartFrame"] + np.array(trivial_twelve(row["DurationFrame"]))).tolist(), 1 / 12)

    assert result == 0


Extract sequences: 0it [00:00, ?it/s]

../SZTR/video/SZTRA103a17.mov ../data/sequences/SZTR_video_SZTRA103a17_00_00_09_TP.mp4 [225, 237, 249, 261, 273, 285, 297, 309, 321, 333, 345, 357, 369, 381, 393, 405, 417, 429, 441, 453, 465, 477, 489, 501, 513, 525, 537, 549, 561, 573, 585, 597, 609, 621, 633, 645, 657, 669, 681, 693, 705, 717, 729, 741, 753, 765, 777, 789, 801, 813, 825, 837, 849, 861, 873, 885, 897, 899] 0.08333333333333333





FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/36.0 [00:00<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/36.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/36.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/36.0 [00:02<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264: 38.0segment [00:02, 15.10segment/s]            
Extract sequences: 1it [00:25, 25.02s/it]

../SZTR/video/SZTRA203a17.mov ../data/sequences/SZTR_video_SZTRA203a17_00_00_10_TP.mp4 [250, 262, 274, 286, 298, 310, 322, 334, 346, 358, 370, 382, 394, 406, 418, 430, 442, 454, 466, 478, 490, 502, 514, 526, 538, 550, 562, 574, 586, 598, 610, 622, 634, 646, 658, 670, 682, 694, 706, 718, 730, 742, 754, 766, 778, 790, 802, 814, 826, 838, 850, 862, 874, 886, 898, 899] 0.08333333333333333





FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/36.0 [00:00<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/36.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/36.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264: 38.0segment [00:02, 18.12segment/s]            
Extract sequences: 2it [00:32, 14.41s/it]

...




FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/82.0 [00:00<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:00<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:02<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:02<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   1%|          | 1.0/82.0 [00:03<00:40,  2.00segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   2%|▏         | 2.0/82.0 [00:03<00:40,  1.99segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   4%|▎         | 3.0/82.0 [00:04<00:57,  1.

../SZTR/video/SZTRA202a10.mov ../data/sequences/SZTR_video_SZTRA202a10_00_00_08_TP.mp4 [200, 212, 224, 236, 248, 260, 272, 284, 296, 308, 320, 332, 344, 356, 368, 380, 392, 404, 416, 428, 440, 452, 464, 476, 488, 500, 512, 524, 536, 548, 560, 572, 584, 596, 608, 620, 632, 644, 656, 668, 680, 692, 704, 716, 728, 740, 752, 764, 776, 788, 800, 812, 824, 836, 848, 860, 872, 884, 896, 908, 920, 932, 944, 956, 968, 980, 992, 1004, 1016, 1028, 1040, 1052, 1064, 1076, 1088, 1100, 1112, 1124, 1136, 1148, 1160, 1172, 1184, 1196, 1208, 1220, 1232, 1244, 1256, 1268, 1280, 1292, 1299] 0.08333333333333333





FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/82.0 [00:00<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:02<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:02<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   1%|          | 1.0/82.0 [00:03<00:40,  2.00segment/s]


FFMPEG scale down to 224, select frames and encode libx264: 86.0segment [00:05, 16.73segment/s]                    
Extract sequences: 9it [01:05,  5.21s/it]

../SZTR/video/SZTRA202a10.mov ../data/sequences/SZTR_video_SZTRA202a10_00_00_00_FP.mp4 [0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156, 168, 180, 192, 204, 216, 228, 240, 252, 264, 276, 288, 300, 312, 324, 336, 348, 360, 372, 384, 396, 408, 420, 432, 444, 456, 468, 480, 492, 504, 516, 528, 540, 552, 564, 576, 588, 600, 612, 624, 636, 648, 660, 672, 684, 696, 708, 720, 732, 744, 756, 768, 780, 792, 804, 816, 828, 840, 852, 864, 876, 888, 900, 912, 924, 936, 948, 960, 972, 984, 996, 1008, 1020, 1032, 1044, 1056, 1068, 1080, 1092, 1104, 1116, 1128, 1140, 1152, 1164, 1176, 1188, 1200, 1212, 1224, 1236, 1248, 1260, 1272, 1284, 1296, 1308, 1320, 1332, 1344, 1356, 1368, 1380, 1392, 1404, 1416, 1428, 1440, 1452, 1464, 1476, 1488, 1500, 1512, 1524, 1536, 1548, 1560, 1572, 1584, 1596, 1608, 1620, 1632, 1644, 1656, 1668, 1680, 1692, 1704, 1716, 1728, 1740, 1752, 1764, 1776, 1788, 1800, 1812, 1824, 1836, 1848, 1860, 1872, 1884, 1896, 1908, 1920, 1924] 0.08333333333333333





FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/82.0 [00:00<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:00<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:01<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:02<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:02<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/82.0 [00:03<?, ?segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   1%|          | 1.0/82.0 [00:03<00:40,  1.99segment/s]


FFMPEG scale down to 224, select frames and encode libx264:   2%|▏         | 2.0/82.0 [00:04<01:03,  1.26segmen